Most libraries built for multi-dimensional array operations can be used to implement tensor networks:

* Numpy (Python)
* PyTorch (Python)
* TensorFlow (Python)
* iTensor (C++, Julia)

There are also libraries such as TensorNetwork which are backend-agnostic wrappers that allow you to utilize tensor network formalisms when working with e.g. Numpy. I've found that the performance of these wrapper isn't good enough yet to use for ML. 

# Training an MPS classifier on MNIST using TensorFlow

In [1]:
# Import the TensorFlow libraries that we will use
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential

## Implementing the product-state feature map

In [2]:
# Inherit from the Keras Layer class
class ProductMap(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def build(self, input_shape):
        pass
        
    def call(self, inputs):
        # Embed each feature "x" into vector [1 - x, x] and stack into a batch x features x 2 array
        output_1 = tf.vectorized_map(lambda x: 1-x, inputs)
        output_2 = tf.vectorized_map(lambda x: x, inputs)
        feature_matrix = tf.stack([output_1, output_2], axis = 2)
        return feature_matrix

## Impementing the MPS layer

![title](mps_4.png)

In [3]:
class MPS_Layer(layers.Layer):

    def __init__(self, num_sites, bond_dim, num_output, **kwargs):
        super().__init__(**kwargs)
        self.n_half = num_sites // 2
        
        # Create variables for tensors to the left, right, and center of output site
        self.left = self.add_weight("left", shape = [2, self.n_half, bond_dim, bond_dim])
        self.right = self.add_weight("right", shape = [2, self.n_half, bond_dim, bond_dim])
        self.middle = self.add_weight("middle", shape = [num_output, bond_dim, bond_dim])

    def call(self, inputs):
        left = tf.einsum("slij,bls->lbij", self.left, inputs[:, :self.n_half]) # Contract left tensors with data
        right = tf.einsum("slij,bls->lbij", self.right, inputs[:, self.n_half:]) # Contract right tensors with data
        left = self.reduction(left) # Contract left tensors together
        right = self.reduction(right) # Contract right tensors together
        outputs = tf.einsum("bij,cjk,bki->bc", left, self.middle, right) # Contract left and right with center
        return outputs

    @staticmethod
    def reduction(tensor):
        # Takes a vector of matrices and contracts them together as a single product
        size = int(tensor.shape[0])
        while size > 1:
            half_size = size // 2
            nice_size = 2 * half_size
            leftover = tensor[nice_size:]
            tensor = tf.matmul(tensor[0:nice_size:2], tensor[1:nice_size:2])
            tensor = tf.concat([tensor, leftover], axis = 0)
            size = half_size + int(size % 2 == 1)
        return tensor[0]

## Loading the MNIST dataset

In [4]:
# Images are retrieved, flattened, and normalized to the range [0, 1], labels are one-hot encoded
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape([60000, 28*28]) / 255
x_test = x_test.reshape([10000, 28*28]) / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

## Building the model

In [5]:
num_features = x_train.shape[1]
bond_dim = 20

# Combine the ProductMap and MPS Layers into a sequential Keras model
model = Sequential()
model.add(ProductMap())
model.add(MPS_Layer(num_features, bond_dim, 10))

# Compile
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
      optimizer=tf.keras.optimizers.RMSprop(0.0001),
      metrics=['accuracy'],
      run_eagerly = False)

## Training the model

In [6]:
epochs = 12
batch_size = 128

model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          validation_data = (x_test, y_test))
score = model.evaluate(x_test, y_test, verbose = 0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 12s 21ms/step - loss: 2.3026 - accuracy: 0.0986 - val_loss: 2.3026 - val_accuracy: 0.0980
Epoch 2/12
469/469 [==============================] - 10s 21ms/step - loss: 2.3026 - accuracy: 0.0999 - val_loss: 2.3026 - val_accuracy: 0.0980
Epoch 3/12
469/469 [==============================] - 10s 21ms/step - loss: 2.3026 - accuracy: 0.0975 - val_loss: 2.3026 - val_accuracy: 0.0980
Epoch 4/12
469/469 [==============================] - 10s 21ms/step - loss: 2.3026 - accuracy: 0.1000 - val_loss: 2.3026 - val_accuracy: 0.0980
Epoch 5/12
469/469 [==============================] - 10s 21ms/step - loss: 2.3026 - accuracy: 0.0985 - val_loss: 2.3026 - val_accuracy: 0.0980
Epoch 6/12
 88/469 [====>.........................] - ETA: 7s - loss: 2.3026 - accuracy: 0.1013

KeyboardInterrupt: 

## Well that didn't work...

In [22]:
class MPS_Layer_Improved(layers.Layer):

    def __init__(self, num_sites, bond_dim, num_output, **kwargs):
        super().__init__(**kwargs)
        self.n_half = num_sites // 2
        
        # Create variables for tensors to the left, right, and center of output site
        self.left = self.add_weight("left", initializer = self.initializer, shape = [2, self.n_half, bond_dim, bond_dim])
        self.right = self.add_weight("right", initializer = self.initializer, shape = [2, self.n_half, bond_dim, bond_dim])
        self.middle = self.add_weight("middle", initializer = self.initializer, shape = [1, num_output, bond_dim, bond_dim])[0]

    def call(self, inputs):
        left = tf.einsum("slij,bls->lbij", self.left, inputs[:, :self.n_half]) # Contract left tensors with data
        right = tf.einsum("slij,bls->lbij", self.right, inputs[:, self.n_half:]) # Contract right tensors with data
        left = self.reduction(left) # Contract left tensors together
        right = self.reduction(right) # Contract right tensors together
        outputs = tf.einsum("bij,cjk,bki->bc", left, self.middle, right) # Contract left and right with center
        return outputs

    @staticmethod
    def initializer(shape, dtype):
        # Tensors need to be initialized such that they basically act like the identity
        (phys_dim, num_sites, bond_dim, bond_dim) = shape
        matrices = phys_dim * num_sites*[tf.eye(bond_dim)]
        weights = tf.reshape(tf.stack(matrices_2), [phys_dim, num_sites, bond_dim, bond_dim])
        noised = weights + tf.random.normal(weights.shape, 0, 1e-2)
        return noised
    
    @staticmethod
    def reduction(tensor):
        # Takes a vector of matrices and contracts them together as a single product
        size = int(tensor.shape[0])
        while size > 1:
            half_size = size // 2
            nice_size = 2 * half_size
            leftover = tensor[nice_size:]
            tensor = tf.matmul(tensor[0:nice_size:2], tensor[1:nice_size:2])
            tensor = tf.concat([tensor, leftover], axis = 0)
            size = half_size + int(size % 2 == 1)
        return tensor[0]

## Let's try this again!

In [23]:
num_features = x_train.shape[1]
bond_dim = 20

# Combine the ProductMap and MPS Layers into a sequential Keras model
model = Sequential()
model.add(ProductMap())
model.add(MPS_Layer_Improved(num_features, bond_dim, 10))

# Compile
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
      optimizer=tf.keras.optimizers.RMSprop(0.0001),
      metrics=['accuracy'],
      run_eagerly = False)

In [25]:
epochs = 25
batch_size = 128

model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          validation_data = (x_test, y_test))
score = model.evaluate(x_test, y_test, verbose = 0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
469/469 [==============================] - 10s 20ms/step - loss: 0.0290 - accuracy: 0.9912 - val_loss: 0.0844 - val_accuracy: 0.9786
Epoch 2/25
469/469 [==============================] - 10s 21ms/step - loss: 0.0278 - accuracy: 0.9914 - val_loss: 0.0808 - val_accuracy: 0.9825
Epoch 3/25
469/469 [==============================] - 10s 21ms/step - loss: 0.0238 - accuracy: 0.9926 - val_loss: 0.1130 - val_accuracy: 0.9778
Epoch 4/25
469/469 [==============================] - 10s 21ms/step - loss: 0.0245 - accuracy: 0.9927 - val_loss: 0.1083 - val_accuracy: 0.9753
Epoch 5/25
469/469 [==============================] - 10s 21ms/step - loss: 0.0220 - accuracy: 0.9935 - val_loss: 0.1030 - val_accuracy: 0.9807
Epoch 6/25
469/469 [==============================] - 10s 21ms/step - loss: 0.0207 - accuracy: 0.9940 - val_loss: 0.1322 - val_accuracy: 0.9762
Epoch 7/25
469/469 [==============================] - 10s 21ms/step - loss: 0.0210 - accuracy: 0.9937 - val_loss: 0.0785 - val_accuracy: